In [57]:
import socket
IP = '127.0.0.1'
PORT = 4242
HANDSHAKE = 'READY'

# ? Create socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.settimeout(1)

data = ''

while (data == ''):
	try:
		sock.sendto(HANDSHAKE.encode('UTF-8'), (IP, PORT))
		data, addr = sock.recvfrom(1024)
		data = data.decode('UTF-8')
	except:
		pass

# TODO Loop other messages

print('HANDSHAKE SENT')

# ? Skip garbage
while (data != 'MSG_START'):
	data, addr = sock.recvfrom(1024)
	data = data.decode('UTF-8')

# ? Parse position
data, addr = sock.recvfrom(1024)
data = data.decode('UTF-8')
dt = data[data.find('\n')+1:]
posX = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
posY = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
posZ = dt[:dt.find('\n')]
# print('posX: ', posX)
# print('posY: ', posY)
# print('posZ: ', posZ)

# ? Parse Speed
data, addr = sock.recvfrom(1024)
data = data.decode('UTF-8')
dt = data[data.find('\n')+1:]
s = dt[:dt.find('\n')]
# print('Speed: ', s)

# ? Parse Acceleration
data, addr = sock.recvfrom(1024)
data = data.decode('UTF-8')
dt = data[data.find('\n')+1:]
aX = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
aY = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
aZ = dt[:dt.find('\n')]
# print('aX: ', aX)
# print('aY: ', aY)
# print('aZ: ', aZ)

# ? Parse Direction
data, addr = sock.recvfrom(1024)
data = data.decode('UTF-8')
dt = data[data.find('\n')+1:]
dX = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
dY = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
dZ = dt[:dt.find('\n')]
# print('dX: ', dX)
# print('dY: ', dY)
# print('dZ: ', dZ)

# ? Get last message
data, addr = sock.recvfrom(1024)

HANDSHAKE SENT


In [58]:
import numpy as np
import math as mt

# ? Position
P = [[posX],
     [posY],
     [posZ]]

P = np.asarray(P).astype('float32')

# ? Acceleration
A = [[aX],
     [aY],
     [aZ]]

A = np.asarray(A).astype('float32')

# ? Direction as a vector
D = [[mt.cos(float(dX)) * mt.cos(float(dY))],
     [mt.sin(float(dX)) * mt.cos(float(dY))],
     [mt.sin(float(dY))]]

D = np.array(D)

s = float(s) / 3.6

# ? time delta in seconds
t = 0.01

# ? let be F be the estimated final position matrix
# ? let noise_corr be the correction of the additive noise caused by sensors incertainity
# ? formula should be
# ? F = P + s * D * t + 0.5 * A * t ** 2 + noise_corr(P, s, D, A)


# ? Calculating final position
# F = P + s * D * t + float(0.5) * A * t ** float(2)

F = P + s * D * t + 0.5 * A * t ** 2

POS = str(F[0][0]) + ' ' + str(F[1][0]) + ' ' + str(F[2][0])
# print(POS)

sock.sendto(POS.encode('UTF-8'), (IP, PORT))

HIST = []

i = 0
E_HIST = []
MSE = 0
# TODO Find a great cofficient to correct MSE
# TODO Maybe correct speed by GPS true position if I find a way
coef = 0.5
while (True):

	# ? Skip garbage
	while (data != 'MSG_START'):
		data, addr = sock.recvfrom(1024)
		data = data.decode('UTF-8')

	turbo = True

	# ? Parse position
	data, addr = sock.recvfrom(1024)
	data = data.decode('UTF-8')
	if (data.find('POSITION') != -1):
		turbo = False
		dt = data[data.find('\n')+1:]
		posX = dt[:dt.find('\n')]
		dt = dt[dt.find('\n')+1:]
		posY = dt[:dt.find('\n')]
		dt = dt[dt.find('\n')+1:]
		posZ = dt[:dt.find('\n')]
		# print('posX: ', posX)
		# print('posY: ', posY)
		# print('posZ: ', posZ)
	else:
		turbo = True

	# ? Parse Speed
	# data, addr = sock.recvfrom(1024)
	# data = data.decode('UTF-8')
	# dt = data[data.find('\n')+1:]
	# s = dt[:dt.find('\n')]
	# print('Speed: ', s)

	# ? Parse Acceleration
	if (turbo == False):
		data, addr = sock.recvfrom(1024)
		data = data.decode('UTF-8')
	dt = data[data.find('\n')+1:]
	aX = dt[:dt.find('\n')]
	dt = dt[dt.find('\n')+1:]
	aY = dt[:dt.find('\n')]
	dt = dt[dt.find('\n')+1:]
	aZ = dt[:dt.find('\n')]
	# print('aX: ', aX)
	# print('aY: ', aY)
	# print('aZ: ', aZ)

	# ? Parse Direction
	data, addr = sock.recvfrom(1024)
	data = data.decode('UTF-8')
	dt = data[data.find('\n')+1:]
	dX = dt[:dt.find('\n')]
	dt = dt[dt.find('\n')+1:]
	dY = dt[:dt.find('\n')]
	dt = dt[dt.find('\n')+1:]
	dZ = dt[:dt.find('\n')]
	# print('dX: ', dX)
	# print('dY: ', dY)
	# print('dZ: ', dZ)

	# ? Position
	if (turbo == False):
		BUF = [[posX],
			[posY],
			[posZ]]

		# B = np.array(BUF)
		# E = np.subtract(P, B)
		# E_HIST.append(E)
		# TODO Calculate MSE
		# P = B.astype('float32')

	# ? Acceleration
	A = [[aX],
		[aY],
		[aZ]]

	A = np.asarray(A).astype('float32')

	# ? Direction as a vector
	D = [[mt.cos(float(dX)) * mt.cos(float(dY))],
		[mt.sin(float(dX)) * mt.cos(float(dY))],
		[mt.sin(float(dY))]]

	D = np.array(D)

	D = D / np.linalg.norm(D)

	# ? let be F be the estimated final position matrix
	# ? let noise_corr be the correction of the additive noise caused by sensors incertainity
	# ? formula should be
	# ? F = P + s * D * t + 0.5 * A * t ** 2 + noise_corr(P, s, D, A)

	# ? Getting and history of positions
	# TODO maybe add a way to know which value is given by GPS
	HIST.append(P)

	# ? Calculating final position
	F = P + s * D * t + 0.5 * A * t ** 2 - coef * MSE

	# ? Sending result
	POS = str(F[0][0]) + ' ' + str(F[1][0]) + ' ' + str(F[2][0])
	# print('itt: ' + str(i) + ' POS: ' + str(POS))

	# ? Updating variables
	DEP = np.subtract(F, P)
	# print("P: " + str(P))
	# print("F: " + str(F))
	s = np.linalg.norm(DEP) * 1/t
	# print("s: " + str(s))
	P = F
	i += 1

# TODO Add Kalman Filter on MQE every 3 sec

	sock.sendto(POS.encode('UTF-8'), (IP, PORT))

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')